In [ ]:
import gzip
import pandas as pd
import numpy as np
import json
import re

In [ ]:
"""
轉換json至dataframe的作法
"""

def json2df(file_path):
    #f = open(r"C:\Users\User\Documents\GitHub\ShareFolder\train_data\NodeAttr.json", 'r')
    # Open the file in read mode
    with open(file_path, 'r') as f:
        # Read the JSON data from the file
        s = f.read().strip()
        l = eval(s)
        json.dumps(l)

    # Convert the JSON data to a DataFrame
    df = pd.read_json(l, orient="index", encoding="utf-8-sig")

    return df


def json2df_2(file_path):
    # Read the JSON file into a pandas DataFrame using the specified parameters
    df = pd.read_json(file_path, orient="index", encoding="utf-8-sig")
    return df



## Download json file and transfer to dataframe

In [ ]:
df_spot = json2df_2(r"C:\Users\User\Documents\GitHub\ShareFolder\osmData\gnn_train_data_osm\spot_20230826.json")
print(df_spot.shape)

df_road = json2df_2(r"C:\Users\User\Documents\GitHub\ShareFolder\osmData\gnn_train_data_osm\road_20230826.json")
print(df_road.shape)

(45673, 432)
(42966, 601)


In [ ]:
df_spot

,location,building,addr:housenumber,addr:postcode,addr:city,addr:street,addr:country,building:levels,payment:coins,amenity,...,stroller,addr:old_housenumber,old_operator,brothel:escort_services,brothel:apartment,source:height,max_level,shelter,generator:source,generator:output:electricity
0,"{'y': {'0': 48.792345000000005, '1': 48.792313...",yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'y': {'0': 48.792336899999995, '1': 48.792451...",yes,55,70190.0,Stuttgart,Stöckachstraße,DE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'y': {'0': 48.792504900000004, '1': 48.792536...",yes,216A,70190.0,Stuttgart,Neckarstraße,NaN,5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'y': {'0': 48.8035822, '1': 48.8036214, '2': ...",yes,59,70469.0,Stuttgart,Rusenschloßstraße,DE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'y': {'0': 48.80472200000001, '1': 48.8046812...",yes,4,70469.0,Stuttgart,Günzelburgweg,DE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45668,"{'y': {'0': 48.792623000000006, '1': 48.792611...",yes,63,70190.0,Stuttgart,Stöckachstraße,DE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45669,"{'y': {'0': 48.7919686, '1': 48.79189380000000...",yes,26,70190.0,NaN,Metzstraße,DE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45670,"{'y': {'0': 48.79285579999999, '1': 48.7929292...",yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45671,"{'y': {'0': 48.79285380000001, '1': 48.7928719...",yes,222,70190.0,Stuttgart,Neckarstraße,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### the object type in dataframe can be tranfered to dictionary type

because spot json file has mistake on long and lan with x and y, I need to exchange it
and when I changed directly in spot json file, it only changed the first 20000 raws.
So, I need to change it manually here.

In [ ]:
see = 19999
print("x is:", df_spot.iloc[see]["location"]['x'])
print("y is:", df_spot.iloc[see]["location"]['y'])


def swap_coordinates(location):
    if 'x' in location and 'y' in location:
        location['x'], location['y'] = location['y'], location['x']
    return location

df_spot.iloc[20000:]['location'] = df_spot.iloc[20000:]['location'].apply(swap_coordinates)


x is: {'0': 9.1586264, '1': 9.1588262, '2': 9.1588877, '3': 9.1586885, '4': 9.1586264}
y is: {'0': 48.78298170000001, '1': 48.7831253, '2': 48.783087599999995, '3': 48.7829468, '4': 48.78298170000001}


C:\Users\User\AppData\Local\Temp\ipykernel_7692\1699253211.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_spot.iloc[20000:]['location'] = df_spot.iloc[20000:]['location'].apply(swap_coordinates)


check if every y (represents long) smaller than x (represents lat)

In [ ]:
see = 14
print("x is:", df_spot.iloc[see]["location"]['x'])
print("y is:", df_spot.iloc[see]["location"]['y'])

def compare_coordinates(location):
    try:
        return location['x']['0'] > location['y']['0']
    except KeyError:
        return np.nan

bool_series = df_spot['location'].apply(compare_coordinates)
print(sum(bool_series))

x is: {'0': 9.1752975, '1': 9.1753072, '2': 9.1751971, '3': 9.1751341, '4': 9.1752429, '5': 9.1752451, '6': 9.1752234, '7': 9.1752005, '8': 9.1751841, '9': 9.175176, '10': 9.1751773, '11': 9.1751878, '12': 9.1752062, '13': 9.175228699999998, '14': 9.175254, '15': 9.1752794, '16': 9.1752975}
y is: {'0': 48.80516370000001, '1': 48.805182099999996, '2': 48.80520709999999, '3': 48.80508680000001, '4': 48.80506210000001, '5': 48.80506629999999, '6': 48.80506989999999, '7': 48.805080000000004, '8': 48.8050946, '9': 48.80511190000001, '10': 48.80513, '11': 48.80514679999999, '12': 48.805160300000004, '13': 48.80516860000001, '14': 48.80517170000002, '15': 48.80516950000001, '16': 48.80516370000001}
25673


### delete the columns whose value is nan

In [ ]:
a = df_spot.iloc[[1]]
print(type(a))
print(a)
colWithNan = a.columns[a.isna().any()]

a_clean = a.drop(colWithNan, axis=1)

<class 'pandas.core.frame.DataFrame'>
                                            location building  \
1  {'y': {'0': 48.792336899999995, '1': 48.792451...      yes   

  addr:housenumber  addr:postcode  addr:city     addr:street addr:country  \
1               55        70190.0  Stuttgart  Stöckachstraße           DE   

  building:levels payment:coins amenity  ... stroller addr:old_housenumber  \
1             NaN           NaN     NaN  ...      NaN                  NaN   

  old_operator brothel:escort_services brothel:apartment source:height  \
1          NaN                     NaN               NaN           NaN   

  max_level shelter generator:source generator:output:electricity  
1       NaN     NaN              NaN                          NaN  

[1 rows x 432 columns]


# Create ttl file

In [ ]:
"""
ttl_file.write("ex:JohnDoe a ex:Person ;\n")
ttl_file.write("ex:hasName 'John Doe' ;\n")
ttl_file.write("ex:hasAge 30 ;\n")
ttl_file.write("ex:hasChild ex:MaryDoe .\n")

ttl_file.write("ex:MaryDoe a ex:Person ;\n")
ttl_file.write("ex:hasName 'Mary Doe' ;\n")
ttl_file.write("ex:hasAge 30 .\n")

ttl_file.write("ex:JannyBoo a ex:Person ;\n")
ttl_file.write("ex:hasName 'Janny Boo' ;\n")
ttl_file.write("ex:hasAge 41 .\n")
"""

'\nttl_file.write("ex:JohnDoe a ex:Person ;\n")\nttl_file.write("ex:hasName \'John Doe\' ;\n")\nttl_file.write("ex:hasAge 30 ;\n")\nttl_file.write("ex:hasChild ex:MaryDoe .\n")\n\nttl_file.write("ex:MaryDoe a ex:Person ;\n")\nttl_file.write("ex:hasName \'Mary Doe\' ;\n")\nttl_file.write("ex:hasAge 30 .\n")\n\nttl_file.write("ex:JannyBoo a ex:Person ;\n")\nttl_file.write("ex:hasName \'Janny Boo\' ;\n")\nttl_file.write("ex:hasAge 41 .\n")\n'

In [ ]:
simplify_r = 0
simplify_s = 0

ttl_file_name = r"
ttl_file = gzip.open(ttl_file_name, mode="wt")
ttl_file.write("@prefix ex: <http://example.org/> .\n")

"""
ttl_file.write("ex:Osm a rdfs:Class .\n")

ttl_file.write("ex:Road a rdfs:Class ;\n")
ttl_file.write("rdfs:subClassOf ex:Osm .\n")

ttl_file.write("ex:Spot a rdfs:Class ;\n")
ttl_file.write("rdfs:subClassOf ex:Osm .\n")
"""
ttl_file.write(f"ex:osm a ex:OSM; \n")
ttl_file.write(f"ex:has_road ex:road; \n")
ttl_file.write(f"ex:has_spot ex:spot. \n") ######################################################################
ttl_file.write(f"ex:road a ex:OSM .\n")
ttl_file.write(f"ex:spot a ex:OSM .\n")


# ttl_file.write(f"ex:{i}_road_pts_{id} a ex:Road_Geo_pts ;\n")

for i in range(df_road.shape[0]-simplify_r):
    df_each = df_road.iloc[[i]]
    colWithNan = df_each.columns[df_each.isna().any()]
    df_each_clean = df_each.drop(colWithNan, axis=1)
    ttl_file.write(f"ex:roadID_{i} a ex:Road ;\n")  ##########################################################
    ttl_file.write(f"ex:has_road ex:road ;\n")
    ttl_file.write(f"ex:geo_info ex:roadID_{i}_location ;\n")
    col_list = list(df_each_clean.columns)
    #print(col_list[0])
    #print(col_list)
    for ind, col in enumerate(col_list):
        # the last one
        if col == col_list[-1]:
            cleaned_text = re.sub(r"[^a-zA-Z0-9]", "", str(df_each_clean[col].values[0]))
            col = str(col).replace(":", "_")
            cleaned_col = re.sub(r"[^a-zA-Z0-9_]", "", col)
            ttl_file.write(f"ex:{cleaned_col} '{cleaned_text}' .\n")
            #ttl_file.write("# ----------------------------------the next one\n")

        # location dict
        elif col == "location":
            continue

        else:
            cleaned_text = re.sub(r"[^a-zA-Z0-9]", "", str(df_each_clean[col].values[0]))
            col = str(col).replace(":", "_")
            cleaned_col = re.sub(r"[^a-zA-Z0-9_]", "", col)
            ttl_file.write(f"ex:{cleaned_col} '{cleaned_text}' ;\n")
            #print(df_each_clean[col].values[0])


    # write location infomation
    ttl_file.write(f"ex:roadID_{i}_location a ex:Road_Geo ;\n") # build node "road_location_id"

    dict_obj = df_road.iloc[i]["location"] ######## this part is wired


    for j in range(len(dict_obj["x"])):
        if j != (len(dict_obj["x"])-1):
            ttl_file.write(f"ex:road_pts ex:roadID_{i}_pts_{j} ;\n") # build node pts_id
        else:
            ttl_file.write(f"ex:road_pts ex:roadID_{i}_pts_{j} .\n")



    for id, (x_num, y_num) in enumerate(zip(dict_obj["x"], dict_obj["y"])):
        ttl_file.write(f"ex:roadID_{i}_pts_{id} a ex:Road_Geo_pts ;\n")
        ttl_file.write(f"ex:long {dict_obj['x'][str(id)]} ;\n")
        ttl_file.write(f"ex:lat {dict_obj['y'][str(id)]} .\n")

    ttl_file.write("# ----------------------------------the next one\n")

#===========================================================================================

for i in range(df_spot.shape[0]-simplify_s):
    df_each = df_spot.iloc[[i]]
    colWithNan = df_each.columns[df_each.isna().any()]
    df_each_clean = df_each.drop(colWithNan, axis=1)
    ttl_file.write(f"ex:spotID_{i} a ex:Spot ;\n")
    ttl_file.write(f"ex:has_spot ex:spot ;\n")
    ttl_file.write(f"ex:geo_info ex:spotID_{i}_location ;\n")
    col_list = list(df_each_clean.columns)
    #print(col_list[0])
    #print(col_list)
    for ind, col in enumerate(col_list):
        # the last one
        if col == col_list[-1]:
            cleaned_text = re.sub(r"[^a-zA-Z0-9]", "", str(df_each_clean[col].values[0]))
            col = str(col).replace(":", "_")
            cleaned_col = re.sub(r"[^a-zA-Z0-9_]", "", col)
            ttl_file.write(f"ex:{cleaned_col} '{cleaned_text}' .\n")
            #ttl_file.write("# ----------------------------------the next one\n")

        # location dict
        elif col == "location":
            continue

        else:
            cleaned_text = re.sub(r"[^a-zA-Z0-9]", "", str(df_each_clean[col].values[0]))
            col = str(col).replace(":", "_")
            cleaned_col = re.sub(r"[^a-zA-Z0-9_]", "", col)
            ttl_file.write(f"ex:{cleaned_col} '{cleaned_text}' ;\n")
            #print(df_each_clean[col].values[0])


    # write location infomation
    ttl_file.write(f"ex:spotID_{i}_location a ex:Spot_Geo ;\n") # build node "road_location_id"

    dict_obj = df_spot.iloc[i]["location"] ######## this part is wired


    for j in range(len(dict_obj["x"])):
        if j != (len(dict_obj["x"])-1):
            ttl_file.write(f"ex:spot_pts ex:spotID_{i}_pts_{j} ;\n") # build node pts_id
        else:
            ttl_file.write(f"ex:spot_pts ex:spotID_{i}_pts_{j} .\n")



    for id, (x_num, y_num) in enumerate(zip(dict_obj["x"], dict_obj["y"])):
        ttl_file.write(f"ex:spotID_{i}_pts_{id} a ex:Spot_Geo_pts ;\n")
        ttl_file.write(f"ex:long {dict_obj['x'][str(id)]} ;\n")
        ttl_file.write(f"ex:lat {dict_obj['y'][str(id)]} .\n")

    ttl_file.write("# ----------------------------------the next one\n")



ttl_file.close()
print("end")

end


The improvement of the code

In [ ]:
import gzip
import re
import pandas as pd  # Assuming you are using pandas DataFrame

# Sample data - you should replace this with your actual data
df_road = pd.DataFrame()
df_spot = pd.DataFrame()

# Function to clean text
def clean_text(text):
    return re.sub(r"[^a-zA-Z0-9]", "", str(text))

# Function to write triples
def write_triples(ttl_file, df, class_name, simplify):
    for i in range(len(df) - simplify):
        df_each = df.iloc[[i]].dropna(axis=1)
        ttl_file.write(f"ex:itemID_{i} a ex:{class_name} ;\n")
        ttl_file.write(f"ex:has_item ex:item ;\n")
        ttl_file.write(f"ex:geo_info ex:itemID_{i}_location ;\n")

        for col in df_each.columns:
            if col == "location":
                continue
            cleaned_text = clean_text(df_each[col].values[0])
            cleaned_col = clean_text(col.replace(":", "_"))
            ttl_file.write(f"ex:{cleaned_col} '{cleaned_text}' ;\n")

        ttl_file.write(".\n")

# Open the TTL file and write data
with gzip.open("file.ttl.gz", mode="wt") as ttl_file:
    ttl_file.write("@prefix ex: <http://example.org/> .\n")
    write_triples(ttl_file, df_road, "Road", 0)
    write_triples(ttl_file, df_spot, "Spot", 0)
